# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_to = pd.read_csv(file_path)

In [ ]:
to_data.head()

In [ ]:
df_to.head()

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods
expensive_house = to_data[["neighbourhood",'single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house',"average_house_value","shelter_costs_owned","shelter_costs_rented"]].groupby("neighbourhood").mean().sort_values(by='average_house_value',ascending=False)
expensive_house = expensive_house[:10]
expensive_house

In [ ]:
# Calculate the mean number of dwelling types units per year
dwelling_types_year = to_data[["neighbourhood",'single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']].groupby([to_data.index,"neighbourhood"]).sum().reset_index()
dwelling_types_year.set_index("year", inplace=True)
dwelling_types_year[:5]

In [ ]:
# Calculate the average monthly shelter costs for owned and rented dwellings
num_dwelling_type = to_data[['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']].groupby(to_data.index).sum()
num_dwelling_type

In [ ]:
year_2001 = pd.DataFrame(num_dwelling_type.iloc[0,:])
year_2001.rename(columns={2001:'num_dwelling'}, inplace=True)

year_2006 = pd.DataFrame(num_dwelling_type.iloc[1,:])
year_2006.rename(columns={2006:'num_dwelling'}, inplace=True)

year_2011 = pd.DataFrame(num_dwelling_type.iloc[2,:])
year_2011.rename(columns={2011:'num_dwelling'}, inplace=True)

year_2016 = pd.DataFrame(num_dwelling_type.iloc[3,:])
year_2016.rename(columns={2016:'num_dwelling'}, inplace=True)

In [ ]:
avg_house_value = to_data[['average_house_value']].groupby(to_data.index).mean()
avg_house_value

In [ ]:
shelter_avg_costs = to_data[['shelter_costs_owned', 'shelter_costs_rented']].groupby(to_data.index).mean()

shelter_avg_costs

In [ ]:
# Combined data frame
neighbourhood =  to_data[['neighbourhood','single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house',"average_house_value","shelter_costs_owned","shelter_costs_rented"]].groupby("neighbourhood").mean().reset_index()
combined_df = pd.concat([df_to.set_index('neighbourhood'),neighbourhood.set_index('neighbourhood')], axis=1, join='inner').reset_index()
combined_df.head()

In [ ]:
by_neighbourhood = to_data[['neighbourhood','average_house_value']].groupby([to_data.index,"neighbourhood"]).mean().reset_index()
by_neighbourhood.set_index('year', inplace=True)
by_neighbourhood[:10]

In [ ]:
costs_per_neighbourhood = to_data.filter(["neighbourhood","shelter_costs_owned","shelter_costs_rented"], axis =1)
costs_per_neighbourhood

In [ ]:
top_neighbourhood = expensive_house.reset_index()
top_neighbourhood = top_neighbourhood["neighbourhood"].tolist()
top_neighbourhood

In [ ]:
top_neighbourhood_df = costs_per_neighbourhood.query("neighbourhood in @top_neighbourhood")
top_neighbourhood_df

In [ ]:
sb_df = top_neighbourhood_df.reset_index()
sb_df

In [ ]:
year = sb_df['year']
neighbourhood = sb_df['neighbourhood']
shelter_costs_owned = sb_df['shelter_costs_owned']
shelter_costs_rented = sb_df['shelter_costs_rented']

In [ ]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    map_1 = px.scatter_mapbox(
    combined_df,
    lat="lat",
    lon="lon",
    color="average_house_value",
    size="average_house_value",
    color_continuous_scale=px.colors.sequential.Cividis
    )
    return map_1

def create_bar_2001():
    bar_plot = year_2001.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2001',title='Dwelling Types in Toronto in 2001', width=650, height=500, color='red',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2006():
    bar_plot = year_2006.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2006', title='Dwelling Types in Toronto in 2006',width=650, height=500, color='blue',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2011():
    bar_plot = year_2011.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2011', title='Dwelling Types in Toronto in 2011',width=650, height=500, color='orange',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_bar_2016():
    bar_plot = year_2016.hvplot.bar(xlabel='2001',ylabel='Dwelling Types in Toronto in 2016', title='Dwelling Types in Toronto in 2016',width=650, height=500, color='purple',rot=90).opts(yformatter='%.0f2')
    return bar_plot

def create_line_chart_owned():
    line_plot = shelter_avg_costs['shelter_costs_owned'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs", title='Average Monthly Shelter Costs for owned Dwellling in Toronto',color="blue")
    return line_plot

def create_line_chart_rented():
    line_plot = shelter_avg_costs['shelter_costs_rented'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs",title='Average Monthly Shelter Costs for rented Dwellling in Toronto', color="orange")
    return line_plot

def average_house_value():
    avg_house_plot = avg_house_value['average_house_value'].hvplot.line(xlabel="Year", ylabel="Avg Monthly Shelter Costs",title='Average House Value in Toronto', color="Green")
    return avg_house_plot

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    by_neighbourhood_plot = by_neighbourhood.hvplot(groupby='neighbourhood', xlabel='Year', ylabel='Avg House Value').opts(yformatter="%.0f")
    return by_neighbourhood_plot
    
def number_dwelling_types():
    """Number of dwelling types per year"""
    dwell_type_plot = dwelling_types_year.hvplot.bar(groupby="neighbourhood", rot=90,width=650, height=500,xlabel='Year', ylabel='Dwelling Types Units')
    return dwell_type_plot

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    by_neighbourhood_hv = px.bar(by_neighbourhood,x='neighbourhood',y='average_house_value',facet_row=by_neighbourhood.index,title='Average House Values in Toronto per Neighbourhood',height=1200, width=900, hover_data=['average_house_value'], color='average_house_value')
    return by_neighbourhood_hv

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    top_10_plot = expensive_house.hvplot.bar(x='neighbourhood', y='average_house_value', rot=90, height=500,title='Top10 Expensive Neighbourhoods in Toronto', xlabel='Neighbourhood', ylabel='Avg House Value')
    return top_10_plot

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    fig = px.sunburst(sb_df, path=[year, neighbourhood],values=shelter_costs_owned,color=shelter_costs_owned,hover_data=[shelter_costs_rented], color_continuous_scale=['red', 'yellow', 'green'],title='Costs Analysis of Most Expensive Neighbourhood in Toronto per Year')
    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
dashboard_title ="# Real Estate Analysis for Toronto: 2001 to 2016"

dashboard_welcome ="The dashboard presents a visual analysis of historical house values, dwelling types per neighbourhood and dwelling costs in Toronto, Ontario from 2001 to 2016"
dashboard_page = pn.Column(dashboard_title, dashboard_welcome, neighbourhood_map())
yearly_analysis = pn.Column(create_bar_2001(), create_bar_2006(), create_bar_2011(),create_bar_2016())
shelter_home_value = pn.Column(create_line_chart_owned(),create_line_chart_rented(), average_house_value())
house_value_dwelling = pn.Column(average_value_by_neighbourhood(), number_dwelling_types(),average_house_value_snapshot())
welcome_plot = pn.Column(dashboard_title,dashboard_welcome, neighbourhood_map())
expensive_neighbourhood = pn.Column(top_most_expensive_neighbourhoods(), sunburts_cost_analysis())

dashboard_tabs = pn.Tabs(             
                ("Home", dashboard_page),
                ("Yearly Market Analysis", yearly_analysis),
                ("Shelter Costs vs Home Value", shelter_home_value),
                ("Neighbourhood Analysis", house_value_dwelling),
                ("Top Expensive Neighbourhoods", expensive_neighbourhood)
)

## Serve the Panel Dashboard

In [ ]:
dashboard_tabs.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
neighbourhood_map().show()

In [ ]:
#create_bar_chart(data, title, xlabel, ylabel, color)

# Bar chart for 2001
#create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# Bar chart for 2006
#create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# Bar chart for 2011
#create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# Bar chart for 2016
#create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
average_house_value()

In [ ]:
average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
#sunburts_cost_analysis()